In [ ]:
import numpy as np
import pandas as pd
import nibabel as nib
from pathlib import Path
from tqdm import tqdm

# Paths
npy_dir = Path(r"E:\kaggle-rsna-data_processing3\volume_uint8_256")
metadata_csv = Path(r"E:\kaggle-rsna-data_processing3\metadata-new.csv")
output_dir = Path(r"E:\Topcow\nnUNet_raw\Dataset502_INFER\imagesTr")

# Create output directory
output_dir.mkdir(parents=True, exist_ok=True)

# Load metadata
df = pd.read_csv(metadata_csv)

# Three possible orientations (FH = IS for simplicity)
# Format: (dim0, dim1, dim2)
ORIENTATION_TYPES = {
    'FH-AP-RL': ('FH', 'AP', 'RL'),  # Type 0
    'AP-HF-RL': ('AP', 'HF', 'RL'),  # Type 1
    'RL-HF-AP': ('RL', 'HF', 'AP')   # Type 2
}

# Target orientation: LPS = (RL, AP, FH)
TARGET = ('RL', 'AP', 'FH')

def get_orientation_type(dim0, dim1, dim2):
    """Get orientation type from directions"""
    orientation = (dim0, dim1, dim2)
    
    if orientation == ('FH', 'AP', 'RL'):
        return 'FH-AP-RL'
    elif orientation == ('AP', 'HF', 'RL'):
        return 'AP-HF-RL'
    elif orientation == ('RL', 'HF', 'AP'):
        return 'RL-HF-AP'
    else:
        raise ValueError(f"Unknown orientation: {orientation}. Expected one of {list(ORIENTATION_TYPES.keys())}")

def convert_to_lps(volume, orientation_type):
    """
    Convert volume to LPS orientation.
    LPS = (RL, AP, FH)
    
    Args:
        volume: numpy array (dim0, dim1, dim2)
        orientation_type: 'FH-AP-RL', 'AP-HF-RL', or 'RL-HF-AP'
    
    Returns:
        volume in LPS orientation
    """
    if orientation_type == 'FH-AP-RL':
        # Current: (FH, AP, RL)
        # Target:  (RL, AP, FH)
        # Need to swap dim0 and dim2
        volume = np.transpose(volume, (2, 1, 0))
        
    elif orientation_type == 'AP-HF-RL':
        # Current: (AP, HF, RL)
        # Target:  (RL, AP, FH)
        # Need to: move dim2->dim0, dim0->dim1, dim1->dim2
        volume = np.transpose(volume, (2, 0, 1))
        # Flip dim2 to get FH (from HF)
        volume = np.flip(volume, axis=2)
        
    elif orientation_type == 'RL-HF-AP':
        # Current: (RL, HF, AP)
        # Target:  (RL, AP, FH)
        # Need to swap dim1 and dim2
        volume = np.transpose(volume, (0, 2, 1))
        # Flip dim2 to get FH (from HF)
        volume = np.flip(volume, axis=2)
    
    return volume

# Create lookup dictionary
orientation_lookup = {}
for _, row in df.iterrows():
    series_uid = row['SeriesInstanceUID']
    orientation_lookup[series_uid] = {
        'dim0': row['ZDirection'],
        'dim1': row['ColDirection'],
        'dim2': row['RowDirection']
    }

# Process each .npy file
npy_files = list(npy_dir.glob("*.npy"))
print(f"Found {len(npy_files)} .npy files")

conversion_stats = {'FH-AP-RL': 0, 'AP-HF-RL': 0, 'RL-HF-AP': 0}

for npy_file in tqdm(npy_files, desc="Converting volumes"):
    series_uid = npy_file.stem
    
    # Check if we have metadata for this series
    if series_uid not in orientation_lookup:
        print(f"\nWarning: No metadata found for {series_uid}, skipping...")
        continue
    
    # Load volume
    volume = np.load(npy_file)
    
    # Convert to float and normalize to [0, 1]
    volume = volume.astype(np.float32) / 255.0
    
    # Get current orientation
    dirs = orientation_lookup[series_uid]
    
    try:
        orientation_type = get_orientation_type(dirs['dim0'], dirs['dim1'], dirs['dim2'])
        conversion_stats[orientation_type] += 1
    except ValueError as e:
        print(f"\n{e} for {series_uid}")
        print(f"Directions: dim0={dirs['dim0']}, dim1={dirs['dim1']}, dim2={dirs['dim2']}")
        continue
    
    # Convert to LPS
    volume = convert_to_lps(volume, orientation_type)
    
    # Create affine matrix with spacing 1, 1, 1
    # NIfTI uses RAS+ by default, LPS needs sign flips
    affine_lps = np.array([
        [1.0, 0.0, 0.0, 0.0],
        [0.0, 1.0, 0.0, 0.0],
        [0.0, 0.0, 1.0, 0.0],
        [0.0, 0.0, 0.0, 1.0]
    ])
    # Create NIfTI image
    nii_img = nib.Nifti1Image(volume, affine_lps)

    # Save as .nii.gz
    output_path = output_dir / f"{series_uid}_0000.nii.gz"
    nib.save(nii_img, output_path)

print(f"\n\nConversion complete! Files saved to {output_dir}")
print(f"\nConversion statistics:")
print(f"  FH-AP-RL:  {conversion_stats['FH-AP-RL']} files")
print(f"  AP-HF-RL:  {conversion_stats['AP-HF-RL']} files")
print(f"  RL-HF-AP:  {conversion_stats['RL-HF-AP']} files")

Found 4326 .npy files


Converting volumes:   2%|▏         | 78/4326 [00:46<42:30,  1.67it/s]


KeyboardInterrupt: 